In [2]:
import pandas as pd
import yfinance as yf
import re
import numpy as np
from termcolor import colored
import sys


def get_currency(x):
    if pd.notna(x) and x != "":
        return yf.Ticker(x).basic_info["currency"]
    else:
        return ""


class Portfolio:
    def __init__(self, path=sys.path[0] + "/portfolio.csv",):
        self.path = path
        self.df_raw = pd.read_csv(path, usecols=["Investment","Quantity"], index_col=False)
        self.usd_gbp = yf.Ticker("USDGBP=X").history()["Close"][-1]
        self.cash = {"GBP": self.df_raw.iloc[-1]["Quantity"],
                     "USD": round(self.df_raw.iloc[-1]["Quantity"] * (1 / self.usd_gbp), 2)}
        self._init_stocks_df()

    def __repr__(self):
        return self.df_stocks.to_string()
    

    def _init_stocks_df(self):
        self.df_stocks = self.df_raw.copy().iloc[0:-1]

        self.df_stocks["Exchange"] = self.df_stocks["Investment"].apply(lambda x: re.search(r"\((.*?)\:", x).group()[1:-1] if re.search(r"\((.*?)\:", x) else "")
        self.df_stocks["Ticker"] = self.df_stocks["Investment"].apply(lambda x: re.search(r"\:(.*?)\)", x).group()[1:-1] if re.search(r"\((.*?)\:", x) else "")
        self.df_stocks["Ticker"] = self.df_stocks["Ticker"] + self.df_stocks["Exchange"].apply(lambda x: ".L" if x == "LSE" else "")
        
        self.df_stocks["Currency"] = ""
        self.df_stocks["Currency"] = self.df_stocks["Ticker"].where(self.df_stocks["Ticker"] != "").apply(get_currency)


        self.df_stocks["Price Raw"] = self.df_stocks["Ticker"].apply(lambda x: yf.Ticker(x).history()["Close"][-1]).round(2)
        
        self.df_stocks["Price GBP"] = np.nan
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "GBP", self.df_stocks["Price Raw"]).round(2)
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "USD", self.df_stocks["Price Raw"] * self.usd_gbp).round(2)
        self.df_stocks["Price GBP"] = self.df_stocks["Price GBP"].where(self.df_stocks["Currency"] != "GBp", self.df_stocks["Price Raw"] / 100).round(2)
        # self.df_stocks["Price USD"] = (self.df_stocks["Price GBP"] * (1 / self.usd_gbp)).round(2)

        
        self.df_stocks["Price GBP 1D"] = self.df_stocks["Ticker"].apply(lambda x: yf.Ticker(x).history()["Close"][-2])
        self.df_stocks["Price GBP 1D"] = self.df_stocks["Price GBP 1D"].where(self.df_stocks["Currency"] != "GBP", self.df_stocks["Price GBP 1D"]).round(2)
        self.df_stocks["Price GBP 1D"] = self.df_stocks["Price GBP 1D"].where(self.df_stocks["Currency"] != "USD", self.df_stocks["Price GBP 1D"] * self.usd_gbp).round(2)
        self.df_stocks["Price GBP 1D"] = self.df_stocks["Price GBP 1D"].where(self.df_stocks["Currency"] != "GBp", self.df_stocks["Price GBP 1D"] / 100).round(2)


    def show_portoflio_totals(self):
        self.total_gbp = (self.df_stocks["Price GBP"] * self.df_stocks["Quantity"]).sum() + self.cash['GBP']
        total_gbp_1d = ((self.df_stocks['Price GBP 1D'] * self.df_stocks["Quantity"]).sum() + self.cash['GBP']).round(2)
        pct_change = round(100 * (self.total_gbp - total_gbp_1d) / total_gbp_1d, 2)

        if self.total_gbp > total_gbp_1d:
            color = "light_green"
        else:
            color = "light_red"

        print(colored("Portfolio totals:", attrs=("underline", "bold", "blink")))
        print(colored(f"£ {self.total_gbp:,.2f} | {'+' + str(pct_change) if pct_change >= 0 else str(pct_change)}%", color))
        print(colored(f"$ {self.total_gbp * 1/self.usd_gbp:,.2f} | {'+' + str(pct_change) if pct_change >= 0 else str(pct_change)}%", color))


    def show_forecast(self, years):
        print()
        print(f"{years}Y forecast:")
        print(f"£ {round(self.total_gbp * 1.09**years, 2):,.2f}")
        print(f"$ {round(self.total_gbp * 1/self.usd_gbp * 1.09**years, 2):,.2f}")



p = Portfolio()
p.show_portoflio_totals()
# p

Portfolio totals:
£ 38,016.88 | -0.57%
$ 47,324.70 | -0.57%


In [6]:
import yfinance as yf
# x = yf.Ticker("AAPL").history()
x = yf.Ticker("AAPL").history(period="10y")
# for i, val in enumerate(x["Close"].items()):
#     print(i, val)
print(x)
# print(x["Close"])



                                 Open        High         Low       Close  \
Date                                                                        
2014-04-14 00:00:00-04:00   16.417763   16.425941   16.270227   16.410843   
2014-04-15 00:00:00-04:00   16.366497   16.409593   16.085264   16.293829   
2014-04-16 00:00:00-04:00   16.296660   16.392292   16.173661   16.326860   
2014-04-17 00:00:00-04:00   16.357997   16.602109   16.332830   16.513397   
2014-04-21 00:00:00-04:00   16.525983   16.739896   16.482572   16.709383   
...                               ...         ...         ...         ...   
2024-04-08 00:00:00-04:00  169.029999  169.199997  168.240005  168.449997   
2024-04-09 00:00:00-04:00  168.699997  170.080002  168.350006  169.669998   
2024-04-10 00:00:00-04:00  168.800003  169.089996  167.110001  167.779999   
2024-04-11 00:00:00-04:00  168.339996  175.460007  168.160004  175.039993   
2024-04-12 00:00:00-04:00  174.259995  178.360001  174.210007  176.550003   